Motif Expression Levels
---

A quick demonstration of computing the level of over/under-expression per motif, using the motif counts in the different models.

In [1]:
import altair as alt

from neuromotifs import datasets

## The Data
Motif counts

In [2]:
motifs = datasets.load_motifs()
motifs

Loading /Users/eyalgal/Library/CloudStorage/Dropbox/PhD/Code/neuromotifs/src/neuromotifs/data/nmc/motifs.csv.gz..


,motif_1,motif_2,motif_3,motif_4,motif_5,motif_6,motif_7,motif_8,motif_9,motif_10,...,reciprocal_edges,seed,celltype,hcid,structural,regressor,regressor_learning_rate,regressor_max_depth,regressor_n_estimators,regressor_random_state
0,1857,1429,1336,88,47,59,7,1,1,1,...,10,NaN,L5_MC,2,False,NaN,NaN,NaN,NaN,NaN
1,2160,1022,1050,27,40,41,8,0,0,0,...,8,1300.0,L5_MC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
2,2201,1110,1048,26,40,24,10,0,0,1,...,7,1301.0,L5_MC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
3,1807,846,925,14,89,75,8,1,1,0,...,19,1302.0,L5_MC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
4,1787,962,918,10,32,45,4,1,0,0,...,9,1303.0,L5_MC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25000,4823,2539,2542,103,129,147,17,1,0,3,...,18,10495.0,L6_LBC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
25001,4762,2396,2453,76,198,200,23,0,6,1,...,27,10496.0,L6_LBC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
25002,4492,2243,2275,79,126,106,29,1,0,2,...,19,10497.0,L6_LBC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0
25003,4633,2341,2345,67,134,128,18,4,4,1,...,20,10498.0,L6_LBC,2,False,GradientBoostingClassifier,0.01,5.0,500.0,1234.0


## EDA

In [3]:
motifs.groupby(["celltype", "model"]).size().unstack().style.format("{:,}").set_caption(
    "Number of samples"
)

/var/folders/l9/n_7g_6jn18b24tz8qvw1rgd40000gn/T/ipykernel_16696/2451431042.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  motifs.groupby(["celltype", "model"]).size().unstack().style.format("{:,}").set_caption(


model,er,dd,ddz,od,ld,bb
celltype,,,,,,
L5_MC,"1,000","1,000","1,000","1,000","1,000",1
L5_TTPC1,"1,000","1,000","1,000","1,000","1,000",1
L5_TTPC2,"1,000","1,000","1,000","1,000","1,000",1
L6_IPC,"1,000","1,000","1,000","1,000","1,000",1
L6_LBC,"1,000","1,000","1,000","1,000","1,000",1


## Motif Analysis

Compute over/under expressions

In [4]:
# w.r.t. ER
z = (
    motifs.filter(regex="celltype|model|motif_")
    .groupby(["model", "celltype"])
    .pipe(lambda grp: (grp.mean() - grp.mean().loc["er"]) / grp.std().loc["er"])
)
z.style.format("{:.2f}", na_rep="--")

/var/folders/l9/n_7g_6jn18b24tz8qvw1rgd40000gn/T/ipykernel_16696/1194323406.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["model", "celltype"])


In [5]:
z_lf = (
    z.reset_index()
    .melt(id_vars=["model", "celltype"], var_name="motif", value_name="z")
    .assign(motif=lambda df: df.motif.str.replace("motif_", "").astype(int))
)
alt.Chart(z_lf).mark_line().encode(
    x="motif",
    y="z",
    color=alt.Color("model").scale(scheme="spectral"),
    row="celltype",
    shape="model",
    tooltip=["celltype", "model", "z"],
).properties(width=400, height=100).resolve_scale(y="independent")

alt.Chart(...)